In [1]:
# Autor: Abeil Coelho Júnior | Universidade Federal do Espírito Sando | PPGCI | abeilc@hotmail.com
# Data de criação: 01/06/2022
# Descrição: Criar processo semiautomático de avaliação de qualidade de dados das coleções do IBRAM, baseado nas regras de catalogação do CCO. 
# Versão: 1
# Data de modificação: 07/06/2022

In [ ]:
import pandas as pd
import numpy as np
import re
import collections
import time

#from textblob import TextBlob

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)


# Variáveis de apoio
#museus = ["museucasadahera_-_indumentaria", "mhn_-_moedas-de-ouro", "museusolarmonjardim_-_acervo", "museudeartereligiosaetradicional_-_acervo-museologico", "museucasahistoricadealcantara_-_objetos", "museucasadahera_-_acervo-museologico", "museusibramgoias_-_museu-casa-da-boa-morte", "museuregionaldesaojoaodelrei_-_acervo_museologico", "museudainconfidencia_-_acervo-museologico", "museusibramgoias_-_museu-casa-da-princesa", "museusibramgoias_-_museu-das-bandeiras", "museudasmissoes_-_acervo-museologico", "museuregionalcasadosottoni_-_acervo-museologico", "museuvictormeirelles_-_mvm-acervo", "museudoouro_-_acervo", "museudodiamante_-_acervo-museologico", "museudasmissoes_-_acervo-museologico"]
elementos_de_metadados = ["Class", "Creation Location", "Creator", "Date", "Description", "Inscription", "Location", "Materials and Techniques", "Measurements", "Measurements_Altura", "Measurements_diametro", "Measurements_espessura", "Measurements_Largura", "Measurements_peso", "Measurements_profundidade", "Other Descriptive Notes", "Physical Description", "Title", "Work Type"]
chave_vazio = "<-99>"
museus = ["museusibramgoias_-_museu-das-bandeiras", "museudoouro_-_acervo", "museusolarmonjardim_-_acervo", "museudasmissoes_-_acervo-museologico", "museucasadahera_-_indumentaria"]

In [ ]:
def tic():
    global _start_time
    _start_time = time.time()


def tac(etapa):
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec, 60)
    (t_hour, t_min) = divmod(t_min, 60)
    print('O script de {} durou: {}horas:{}min:{}seg'.format(
        etapa, t_hour, t_min, t_sec))

tic()

In [ ]:
acervo_ibram = pd.read_excel("./Acervo_Ibram_Crosswalked.xlsx", index_col=0)
# remover colunas "off"
acervo_ibram = acervo_ibram.loc[:, ~acervo_ibram.columns.str.startswith('Off')]
acervo_ibram


sampled_data = pd.DataFrame()
for museu in museus:
    sample_temp = acervo_ibram.query("Nome_Acervo == '{}'".format(museu))
    sample_temp = sample_temp.sample(n = 10, replace = False)
    frames = [sampled_data, sample_temp]
    sampled_data = pd.concat(frames)
    del sample_temp, frames
sampled_data

In [ ]:
# Carregar base do acervo
acervo_ibram = pd.read_excel("./Acervo_Ibram_Crosswalked.xlsx", index_col=0)
# remover colunas "off"
acervo_ibram = acervo_ibram.loc[:, ~acervo_ibram.columns.str.startswith('Off')]
acervo_ibram

regras_cco = pd.read_excel("../Planilhas de apoio/Base_Regex.xlsx")
print("Regras CCO")
display(regras_cco)
print(regras_cco.shape)

dimencoes = pd.read_excel("../Planilhas de apoio/Dimencoes.xlsx")
print("\nDimensões")
display(dimencoes)
print(dimencoes.shape)

controlados = pd.read_excel("../Planilhas de apoio/Campos_com_vocabularios_controlados.xlsx", index_col=0)
print("\nCampos com vocabulários controlados")
display(controlados)
print(controlados.shape)

In [ ]:
def verificador_multivalor(regex, valores, negativo, tipo):
    respostas = []
    if tipo == "Match":
        for valor in valores:
            match = re.match(regex, valor)
            if match is not None:
                resposta = True    
            else:
                resposta = False
            respostas.append(resposta)
            del resposta
    else:
        for valor in valores:
            match = re.fullmatch(regex, valor)
            if match is not None:
                resposta = True    
            else:
                resposta = False
            respostas.append(resposta)
            del resposta
    
    respostas=collections.Counter(respostas)
    resultado_final = respostas.most_common()[0]
    resultado_final = resultado_final[0]
    
    if negativo == 1:
        resultado_final = not resultado_final
    else:
        pass
        
    return resultado_final

def verificador(regex, valor, negativo, tipo, nome_regra, museu, metadado):
    valor = str(valor)
    
    if valor == '<-99>':
        resposta = False
        return resposta
    else:
        if nome_regra == 'Fazer uso de vocabulário controlado':
            controlados = pd.read_excel("./Campos_com_vocabularios_controlados.xlsx", index_col=0)
            controlados_check = controlados.query("Colecao == '{}' and Campos_Ajutados == '{}'".format(museu, metadado))
            if controlados_check.shape[0] == 0:
                resposta = False
            else:
                resposta = True
            return resposta
        
        if nome_regra == 'Usar o mesmo idioma do catálogo':
            resposta = True
            return resposta
        
        if "|" in valor:
            valores = valor.split("|")
            resposta = verificador_multivalor(regex, valores, negativo, tipo)
            return resposta
        
        if ">" in valor:
            valor = valor.split("> ")[-1]
        
        
            
                
    if tipo == "Match":
        match = re.match(regex, valor)
    else:
        match = re.fullmatch(regex, valor)
        
    if match is not None:
        resposta = True    
    else:
        resposta = False

    if negativo == 1:
        resposta = not resposta

    return resposta

In [ ]:
resultado_colecao = pd.DataFrame()

for museu in museus:
    tic()
#     acervo_avaliado = acervo_ibram.query("Nome_Acervo == '{}'".format(museu))
    acervo_avaliado = sampled_data.query("Nome_Acervo == '{}'".format(museu))
    print("\n\n############################################\n")
    print(museu)
    print("A coleção possui", acervo_avaliado.shape[0],"itens.")
    print("\n############################################\n\n")
    

    acervo_avaliado = acervo_avaliado.fillna("<-99>")
    #acervo_avaliado["Creator"].replace({"Não identificado": "<-99>"}, inplace=True)
    #acervo_avaliado["Creator"].replace({"Não identificada": "<-99>"}, inplace=True)
    #acervo_avaliado["Creator"].replace({"Ignorado": "<-99>"}, inplace=True)
    #acervo_avaliado["Creator"].replace({"Sem Autor": "<-99>"}, inplace=True)


    
    tamanho_total = acervo_avaliado.shape[0]

    # Loop para cada elemento de metadado
    for metadado in elementos_de_metadados:
        print("\n\n",metadado)

        # Selecionar regras aplicaveis ao elemento de metadado
        regras_aplicaveis = regras_cco[regras_cco.iloc[:,1].str.contains(metadado)]

        # Selecionar coluna correspondente ao metadado
        coluna_foco = pd.DataFrame()
        coluna_foco["foco"] = acervo_avaliado[metadado]

        # Loop para cada regra existente
        print("Quantidade de regras:",regras_aplicaveis.shape[0], "\n")
        print("==========================================================================================")
        
        for index_regras, row_regras in regras_aplicaveis.iterrows():
            nome_regra = row_regras.iloc[0]
            regex = row_regras.iloc[2]
            ind_negativo = row_regras.iloc[3]
            tipo = row_regras.iloc[4]

            avaliacoes = []

            print("Nome da regra:",nome_regra,"\nExpressão:", regex,"\nNegação:", ind_negativo, "\nTipo:", tipo)

            # Loop para cada registro presenta na coluna do metadado
            for index_dado, row_dado in coluna_foco.iterrows():
                dado_descricional = row_dado[0]            

                avaliacao = verificador(regex, dado_descricional, ind_negativo, tipo, nome_regra, museu, metadado)
                avaliacoes.append(avaliacao)

            resultados_distintos = list(dict.fromkeys(avaliacoes))


            counter=collections.Counter(avaliacoes)
            print("Os resultados obtidos foram:",dict(counter),"\n")

            resultado_geral = pd.DataFrame({'Avaliacao': avaliacoes, "Dado": coluna_foco["foco"], "Colecao": museu, "Campo_Metadado": metadado, "Regra": nome_regra, "Regex": regex, "Total": tamanho_total})
            frames = [resultado_colecao, resultado_geral]
            resultado_colecao = pd.concat(frames)

            if len(resultados_distintos) == 1:
                print("==> Amostra", resultados_distintos[0],"\n")
                temp = resultado_geral.query("Avaliacao == {}".format(resultados_distintos[0]))
                print(temp["Dado"].sample(),'\n')
                
            else:
                print("==> Amostra", resultados_distintos[0],"\n")
                temp = resultado_geral.query("Avaliacao == {}".format(resultados_distintos[0]))
                print(temp["Dado"].sample(),'\n')

                print("==> Amostra", resultados_distintos[1])                
                temp = resultado_geral.query("Avaliacao == {}".format(resultados_distintos[1]))
                print(temp["Dado"].sample(),'\n')

            del resultado_geral, temp
            print("=======================================")
        del coluna_foco
    tac("Competou a coleção", museu)

In [ ]:
# # resultado_colecao.to_excel("./Resultados_preliminares.xlsx")
print(resultado_colecao.Colecao.unique())
resultado_colecao.shape

In [ ]:
resultado_colecao = resultado_colecao.reset_index()

In [ ]:
resultado_colecoes_data = (resultado_colecao.groupby(["Colecao", "Regra", "Campo_Metadado", "Avaliacao", "Regex", "Total"]).agg(
            resultado=("Avaliacao", "count")).reset_index())

resultado_colecoes_data = resultado_colecoes_data.pivot(
            index=["Colecao", "Campo_Metadado", "Regra", "Total"], columns=["Avaliacao"], values="resultado")
resultado_colecoes_data = resultado_colecoes_data.reset_index()

resultado_colecoes_data = resultado_colecoes_data.rename(columns={False: "0", True: "1"})
resultado_colecoes_data = resultado_colecoes_data.fillna(0)
resultado_colecoes_data

In [ ]:
resultados_preliminares = pd.merge(resultado_colecoes_data, dimencoes, how="left", on="Campo_Metadado")
resultados_preliminares = resultados_preliminares[["Colecao", "Dimensão", "Campo_Metadado", "Regra", "0", "1", "Total"]]
resultados_preliminares = resultados_preliminares.sort_values(by=["Dimensão", "Campo_Metadado"]).reset_index(drop=True)
resultados_preliminares

In [ ]:
resultados_preliminares.to_excel("./Resultados_preliminares.xlsx")